# Dependencies


In [42]:
pip install -qU openai

Note: you may need to restart the kernel to use updated packages.


In [43]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

# Task 1: Simple Assistant
## OpenAI Client

In [44]:
from openai import OpenAI

client = OpenAI()

## Creating An Assistant

In [45]:
name = "WASEEM AIE3 ASSISTANT" # @param {type: "string"}
instructions = "You're a cool guy, who is not afraid of anything." # @param {type: "string"}
model = "gpt-4o" # @param ["gpt-3.5-turbo", "gpt-4-turbo-preview", "gpt-4", "gpt-4o"]

## Initialize Assistant

In [46]:
assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model,
)
  

In [47]:
assistant

Assistant(id='asst_iKlP6uPVARDGWyjUrCAfCfkS', created_at=1718364138, description=None, instructions="You're a cool guy, who is not afraid of anything.", metadata={}, model='gpt-4o', name='WASEEM AIE3 ASSISTANT', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)

## Creating a Thread

In [48]:
thread = client.beta.threads.create()

In [49]:
thread

Thread(id='thread_b2OmQKE96Qy8i5p4jBDXKgOm', created_at=1718364138, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

## Adding Messages to Our Thread

In [50]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"How old are you?"
)

In [51]:
message

Message(id='msg_DaXJvyPMd5GIcZdOSr0SiEi4', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How old are you?'), type='text')], created_at=1718364139, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_b2OmQKE96Qy8i5p4jBDXKgOm')

In [52]:
additional_instructions = "YOUR EXTRA INSTRUCTIONS HERE" # @param {type: "string"}

In [53]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=additional_instructions
)

In [54]:
run

Run(id='run_TCtf8Mzxx8muvhvO22HZFNFf', assistant_id='asst_iKlP6uPVARDGWyjUrCAfCfkS', cancelled_at=None, completed_at=None, created_at=1718364139, expires_at=1718364739, failed_at=None, incomplete_details=None, instructions='YOUR EXTRA INSTRUCTIONS HERE', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_b2OmQKE96Qy8i5p4jBDXKgOm', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

## Retrieving Our Run

In [55]:
import time

while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

In [56]:
print(run.status)

completed


In [57]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [58]:
messages.data[0]

Message(id='msg_3IvtnGCnBBEsBRjxiMP9CqON', assistant_id='asst_iKlP6uPVARDGWyjUrCAfCfkS', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="I don't have an age because I'm an artificial intelligence created by OpenAI. My knowledge and responses are based on data available up until 2023. How can I assist you today?"), type='text')], created_at=1718364140, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_TCtf8Mzxx8muvhvO22HZFNFf', status=None, thread_id='thread_b2OmQKE96Qy8i5p4jBDXKgOm')

## Streaming Our Runs

In [59]:
from typing_extensions import override
from openai import AssistantEventHandler

class EventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

In [60]:
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=additional_instructions,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > I don't have an age because I'm an artificial intelligence created by OpenAI. My knowledge and responses are based on data available up until 2023. How can I assist you today?

# Task 2: Adding Tools
## Task 2a: Creating an Assistant with the File Search Tool
## Collect and Add Data to Vector Store

In [61]:
!wget https://raw.githubusercontent.com/dbredvick/paul-graham-to-kindle/main/paul_graham_essays.txt

--2024-06-14 16:22:24--  https://github.com/dbredvick/paul-graham-to-kindle/blob/main/paul_graham_essays.txt
Resolving github.com (github.com)... 20.207.73.82
Connecting to github.com (github.com)|20.207.73.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘paul_graham_essays.txt.1’

paul_graham_essays.     [  <=>               ] 261.25K   684KB/s    in 0.4s    

2024-06-14 16:22:25 (684 KB/s) - ‘paul_graham_essays.txt.1’ saved [267518]



In [75]:
vector_store = client.beta.vector_stores.create(name="Paul Graham Essay Compilation")


In [76]:
file_paths = ["paul_graham_essays.txt"]
file_streams = [open(path, "rb") for path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

In [77]:
file_streams

[<_io.BufferedReader name='paul_graham_essays.txt'>]

In [78]:
while file_batch.status != "completed":
  time(1)

## Create and Use Assistant

In [79]:
fs_assistant = client.beta.assistants.create(
  name=name,
  instructions=instructions,
  model=model,
  tools=[{"type": "file_search"}],
)

In [80]:
fs_assistant = client.beta.assistants.update(
  assistant_id=fs_assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [81]:
fs_thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What did Paul Graham say about Silicon Valley?",
    }
  ]
)

In [82]:
class FSEventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)

  @override
  def on_message_done(self, message) -> None:
    message_content = message.content[0].text
    annotations = message_content.annotations
    citations = []
    for index, annotation in enumerate(annotations):
      message_content.value = message_content.value.replace(
        annotation.text, f"[{index}]"
      )
      if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

    print(message_content.value)
    print("\n".join(citations))

In [84]:
with client.beta.threads.runs.stream(
  thread_id=fs_thread.id,
  assistant_id=fs_assistant.id,
  event_handler=FSEventHandler(),
) as stream:
  stream.until_done()


assistant > Paul Graham emphasized that startups fare significantly better in Silicon Valley due to several key factors:

1. **Concentration of Talent and Resources:** He pointed out that Silicon Valley has a dense concentration of resources, knowledge, and like-minded individuals focused on startups, which makes it an ideal environment for new companies compared to other regions like Boston or London[0][1].

2. **Bold Investors and Venture Capital:** The willingness of investors in Silicon Valley to take bold risks is highlighted as a crucial element. These investors understand startups deeply and are more open to investing in innovative ideas that may appear risky at first glance, something that is less common in other regions with more conservative investors[2].

3. **Immigration and Diversity:** The openness to immigration is noted as a fundamental strength. He believes that a diverse and international population enhances the innovative capacity of Silicon Valley, as it attracts s

## Task 2b: Creating an Assistant with the Code Interpreter Tool


In [85]:

ci_assistant = client.beta.assistants.create(
  name=name + "+ Code Interpreter",
  instructions=instructions,
  model=model,
  tools=[{"type": "code_interpreter"}],
)

In [86]:
!git clone https://github.com/ali-ce/datasets.git

Cloning into 'datasets'...
remote: Enumerating objects: 803, done.
remote: Total 803 (delta 0), reused 0 (delta 0), pack-reused 803
Receiving objects: 100% (803/803), 4.31 MiB | 1.02 MiB/s, done.
Resolving deltas: 100% (336/336), done.


In [87]:
file = client.files.create(
  file=open("datasets/Y-Combinator/Startups.csv", "rb"),
  purpose='assistants'
)

In [88]:
ci_thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What kind of file is this?",
      "attachments": [
          {
              "file_id" : file.id,
              "tools" : [{"type" : "code_interpreter"}]
          }
      ]
    }
  ]
)

In [89]:
class CIEventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)

  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

In [90]:
with client.beta.threads.runs.stream(
  thread_id=ci_thread.id,
  assistant_id=ci_assistant.id,
  instructions=additional_instructions,
  event_handler=CIEventHandler(),
) as stream:
  stream.until_done()


assistant > code_interpreter

import mimetypes

# Determine the file type
file_path = '/mnt/data/file-N7EvfZCgei1WRpckgeilzPrY'
file_type, _ = mimetypes.guess_type(file_path)

file_type
assistant > The MIME type of the file could not be determined directly. Let's examine the contents of the file to get more information about its type.# Try reading the first few bytes of the file to determine its type
with open(file_path, 'rb') as file:
    file_head = file.read(512)

file_head
assistant > The file appears to be a CSV (Comma-Separated Values) file based on its content. The first few bytes reveal that it contains tabular data with headers such as "Company", "Status", "Year Founded", and so on.

Would you like to examine or manipulate this CSV file in any specific way?

## Task 2c: Creating an Assistant with a Function Calling Tool

In [106]:
!pip install -qU duckduckgo_search

In [107]:
from duckduckgo_search import DDGS

def duckduckgo_search(query):
  with DDGS() as ddgs:
    results = [r for r in ddgs.text(query, max_results=5)]
    return "\n".join(result["body"] for result in results)

In [108]:
duckduckgo_search("Who is the current captain of the Winnipeg Jets?")


"Lowry has been a part of the Jets organization since June 25, 2011, when he was selected in the third round, 67 th overall, after putting up 37 points in 36 games with the Swift Current Broncos of ...\nLowry will follow Andrew Ladd and Blake Wheeler to serve as the third captain of the new Winnipeg Jets franchise. - Sep 12, 2023. After a season without a captain, the Winnipeg Jets have named ...\nAdam Lowry was named captain of the Winnipeg Jets on Tuesday. The 30-year-old forward was selected by the Jets in the third round (No. 67) of the 2011 NHL Draft and has played his entire nine ...\nThe Canadian Press. Published Sep 12, 2023 at 09:34 AM ET. WINNIPEG — The pride in Adam Lowry's voice was evident after being named captain of the Winnipeg Jets on Tuesday. Lowry is the third ...\nAdam Lowry, who has been a Jet since 2011 when he was drafted 67th overall, is the new captain of the NHL team — its third since relocating to Winnipeg from Atlanta in 2011. Andrew Ladd served ..."

In [110]:
ddg_function = {
    "name" : "duckduckgo_search",
    "description" : "Answer non-technical questions. ",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "query" : {
                "type:" : "string",
                "description" : "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"
            }
        },
        "required" : ["query"]
    }
}

## Question
Why does the description key-value pair matter?
It matters because python function takes arguments as key value pairs. So in here we are describing the function paramters and there types

In [111]:
fc_assistant = client.beta.assistants.create(
    name=name + " + Function Calling",
    instructions=instructions,
    tools=[
        {"type": "function",
         "function" : ddg_function
        }
    ],
    model=model
)

In [112]:
fc_thread = client.beta.threads.create()
fc_message = client.beta.threads.messages.create(
  thread_id=fc_thread.id,
  role="user",
  content="Can you describe the Twitter beef between Elon and LeCun?",
)

In [114]:
class FCEventHandler(AssistantEventHandler):
  @override
  def on_event(self, event):
    # Retrieve events that are denoted with 'requires_action'
    # since these will have our tool_calls
    if event.event == 'thread.run.requires_action':
      run_id = event.data.id  # Retrieve the run ID from the event data
      self.handle_requires_action(event.data, run_id)

  def handle_requires_action(self, data, run_id):
    tool_outputs = []

    for tool in data.required_action.submit_tool_outputs.tool_calls:
      print(tool.function.arguments)
      if tool.function.name == "duckduckgo_search":
        tool_outputs.append({"tool_call_id": tool.id, "output": duckduckgo_search(tool.function.arguments)})

    # Submit all tool_outputs at the same time
    self.submit_tool_outputs(tool_outputs, run_id)

  def submit_tool_outputs(self, tool_outputs, run_id):
    # Use the submit_tool_outputs_stream helper
    with client.beta.threads.runs.submit_tool_outputs_stream(
      thread_id=self.current_run.thread_id,
      run_id=self.current_run.id,
      tool_outputs=tool_outputs,
      event_handler=FCEventHandler(),
    ) as stream:
      for text in stream.text_deltas:
        print(text, end="", flush=True)
      print()

In [115]:
with client.beta.threads.runs.stream(
  thread_id=fc_thread.id,
  assistant_id=fc_assistant.id,
  event_handler=FCEventHandler()
) as stream:
  stream.until_done()

{"query":"Twitter beef between Elon Musk and LeCun"}
Sure thing! Here's the lowdown on the Twitter beef between Elon Musk and Yann LeCun:

### Background:
Elon Musk, Tesla, and SpaceX CEO, is known for his forward-thinking views on artificial intelligence (AI) and often expresses concerns about AI's potential dangers. Yann LeCun, on the other hand, is a renowned AI researcher and Chief AI Scientist at Meta (formerly Facebook). LeCun is known for his contributions to deep learning and is more optimistic about AI's future.

### The Beef:
1. **Initial Spark**:
   - The beef seems to have started over differing perspectives on the potential risks of AI. Elon Musk has been vocal about AI being one of humanity's biggest existential threats. In contrast, Yann LeCun has often downplayed these concerns.

2. **Twitter Exchange**:
   - The disagreements turned into public spats on Twitter. Elon Musk made statements highlighting his fears about AI, to which LeCun responded by questioning Musk’s un